In [8]:
import pandas as pd
import numpy as np

file_path = r"../data/at-dataset/Scats_Data.csv"

df = pd.read_csv(file_path, sep="\t")

df[["Detector_ID", "Lane"]] = df["Detector"].str.split("-", expand=True)
df = df.drop(columns=["Detector"])
df["Detector_ID"] = pd.to_numeric(df["Detector_ID"], errors="coerce")
df["Lane"] = pd.to_numeric(df["Lane"], errors="coerce")
df["DateTime"] = pd.to_datetime(df["Date"] + " " + df["Time"], format="%Y-%m-%d %H:%M")

df = df.drop_duplicates(subset=["Detector_ID", "Lane", "DateTime"])
# df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
# df['Time'] = pd.to_datetime(df['Time'], format='%H:%M').dt.time

In [9]:
df.describe()

,Invalid,Volume,Detector_ID,Lane,DateTime
count,1.268714e+06,1.268714e+06,1.268714e+06,1.268714e+06,1268714
mean,5.868234e-01,1.241695e+02,2.184701e+03,7.205200e+00,2024-04-14 22:37:38.667910400
min,0.000000e+00,0.000000e+00,2.013000e+03,1.000000e+00,2023-01-01 00:00:00
25%,0.000000e+00,1.600000e+01,2.061000e+03,3.000000e+00,2023-08-23 20:00:00
50%,0.000000e+00,7.200000e+01,2.257000e+03,6.000000e+00,2024-04-14 06:00:00
75%,0.000000e+00,1.920000e+02,2.257000e+03,1.000000e+01,2024-12-03 03:00:00
max,2.400000e+01,1.498000e+03,2.402000e+03,1.900000e+01,2025-07-31 23:00:00
std,2.590651e+00,1.389751e+02,1.396438e+02,4.670805e+00,NaN


In [11]:
df.groupby(["Detector_ID", "Lane"]).size()

Detector_ID  Lane
2013         1       22197
             2       22197
             3       22197
             4       22197
             5       22197
             6       22197
             7       22197
             8       22197
             9       22197
             10      22197
             11      22197
             12      22197
             13      22197
2061         1       22412
             2       22412
             3       22412
             4       22412
             5       22412
             6       22412
             7       22412
             8       22412
             9       22412
             10      22412
             11      22412
             12      22412
2257         1       22173
             2       22173
             3       22173
             4       22173
             5       22173
             6       22173
             7       22173
             8       22173
             9       22173
             10      22173
             11      22173
          

In [43]:
site_list = df['Detector_ID'].unique()
full_time_index = pd.date_range(df["DateTime"].min(), df["DateTime"].max(), freq="h")

main_df = pd.DataFrame()

for site in site_list:
    lane_list = df.loc[df['Detector_ID'] == site, 'Lane'].unique()
    df_lane_list = pd.DataFrame()
    for lane in lane_list:
        # base hourly frame for this site/lane
        interpolate_df = (
            pd.DataFrame(index=full_time_index)
            .rename_axis('DateTime')
            .reset_index()
        )
        interpolate_df['Detector_ID'] = site
        interpolate_df['Lane'] = lane

        # take the actual observations for this site/lane
        sub = df[(df['Detector_ID'] == site) & (df['Lane'] == lane)][
            ['DateTime', 'Volume']
        ].copy()

        # merge to align values to the hourly grid
        interpolate_df = interpolate_df.merge(
            sub, on='DateTime', how='left'
        )

        # set DateTime index for time-aware interpolation
        interpolate_df = interpolate_df.sort_values('DateTime').set_index('DateTime')

        # time-aware interpolation (add a limit if you want only short gaps filled)
        interpolate_df['Volume'] = interpolate_df['Volume'].interpolate(
            method='time'  #, limit=6
        )

        # back to rows
        interpolate_df = interpolate_df.reset_index()

        df_lane_list = pd.concat([df_lane_list, interpolate_df], ignore_index=True)

    main_df = pd.concat([main_df, df_lane_list], ignore_index=True)

